In [16]:
import os
import time

import pandas as pd
from dotenv import load_dotenv
from sqlalchemy import create_engine

from mods import gmap
from mods import pandas_mod as pdm

In [ ]:
load_dotenv()

username = os.getenv("MYSQL_USERNAME")
password = os.getenv("MYSQL_PASSWORD")
target_ip = os.getenv("MYSQL_IP")
target_port = int(os.getenv("MYSQL_PORTT"))
db_name = os.getenv("MYSQL_DB_NAME")

engine = create_engine(
    f"mysql+pymysql://{username}:{password}@{target_ip}:{target_port}/{db_name}")

sql = "SELECT * FROM salon_place_id"

df = pd.read_sql(sql, engine)

df_edit = df.copy()

In [18]:
# 根據place id抓取資料，並做成新的dataframe
api_key = os.getenv("GMAP_KEY3")

id_list = df_edit["place_id"].values

data = []
for id_ in id_list:
    place_info = gmap.get_place_dict(api_key=api_key, place_id=id_)
    data.append(place_info)
    time.sleep(1)

df_search = pd.DataFrame(data=data)

In [37]:
# 將要合併的欄位取出，並與原本的df合併
df_search = df_search[['place_id', 'address', 'phone', 'opening_hours', 'rating',
                       'rating_total', 'longitude', 'latitude', 'map_url', 'website', 'newest_review']]

df_combine = df_edit.merge(df_search, how="left", on="place_id")

folder = r"C:\Users\add41\Documents\Data_Engineer\Project\example_data\gmap_full_search\temp"
file = "salon_temp.csv"
path = os.path.join(folder, file)
df_combine.to_csv(path, index=False)

In [38]:
# 新增類別
df_combine["category"] = "寵物美容"

In [39]:
# 先取出正常地址格式中的市和區
pattern1 = r"([^\d\s]{2}市)([^\d\s]{1,2}區)"

df_combine[["city", "district"]] = df_combine["address"].str.extract(pattern1)

In [40]:
# 再取出反向地址格式中的市和區
pattern2 = r"([^\d\s]{2}區)([^\d\s]{2}市)"

mask = df_combine["city"].isna() & df_combine["district"].isna()
extracted = df_combine.loc[mask, "address"].str.extract(pattern2)
df_combine.loc[mask, ["district", "city"]] = extracted.values

In [41]:
# 讀取地區資料，並與店家表join，留下loc id，並將city和district欄位去除
sql_loc = "SELECT * FROM location"

df_loc = pd.read_sql(sql_loc, engine)

df_loc_select = df_loc[["loc_id", "city", "district"]]

df_final = df_combine.merge(df_loc_select, how="left", on=["city", "district"])

df_final.drop(columns=["city", "district"], axis=1, inplace=True)

In [42]:
# 讀取類別資料，並與店家表join，留下類別id，將類別欄位去除
sql_catego = "SELECT * FROM Category"

df_catego = pd.read_sql(sql_catego, engine)

df_final = df_final.merge(df_catego, how="left",
                          left_on="category", right_on="category_name")

df_final.drop(columns=["category_name", "category_eng"], axis=1, inplace=True)

In [43]:
# 加上類別id
df_final["id"] = ""

df_final = pdm.reassign_id(df_final, "id", "sal")

In [45]:
op_hours_list = []
for index, row in df_final.iterrows():
    op_time = row["opening_hours"]
    op_hours = pdm.trans_op_time_to_hours(op_time)
    op_hours_list.append(op_hours)

df_final["op_hours"] = op_hours_list
df_final.drop(columns="opening_hours", axis=1, inplace=True)

In [46]:
new_col = ["id", 'name', 'buss_status', 'loc_id', 'address', 'phone', "op_hours", 'category_id', 'rating',
           'rating_total', 'newest_review', 'longitude', 'latitude', 'map_url', 'website', 'place_id', 'update_date']

df_final = df_final[new_col]

In [47]:
df_final

,id,name,buss_status,loc_id,address,phone,op_hours,category_id,rating,rating_total,newest_review,longitude,latitude,map_url,website,place_id,update_date
0,sal0001,HalloPet 寵物精緻沙龍、行為訓練中心,OPERATIONAL,NTP013,236台灣新北市土城區永豐路13巷26號,None,54.0,4,4.6,57,2023-03-03,121.462405,24.988656,https://maps.google.com/?cid=5226194862743386367,None,ChIJwXEpx84CaDQR_3jKwIksh0g,2025/10/30
1,sal0002,毛手毛腳寵物沐浴小舖,OPERATIONAL,NaN,236台灣新北市土城区广福街19號一樓,0222669162,45.0,4,4.5,34,2019-06-28,121.456971,24.989121,https://maps.google.com/?cid=12071886918002509449,https://www.facebook.com/pg/miniu002/posts/,ChIJe0aWAsgCaDQRiYY5PDH1h6c,2025/10/30
2,sal0003,三福寵物,OPERATIONAL,NTP006,231046台灣新北市新店區安德街169號,0222128280,73.0,4,4.1,87,2024-02-22,121.508644,24.959186,https://maps.google.com/?cid=12800348904265973277,None,ChIJo0B_90sCaDQRHX7VX4L5o7E,2025/10/30
3,sal0004,黃金犬寵物美容,OPERATIONAL,NTP006,231台灣新北市新店區永安街1號,0229468985,71.0,4,4.6,11,2021-01-23,121.516563,24.986044,https://maps.google.com/?cid=15698603040988438287,None,ChIJr2_1DA0CaDQRD5_7f0qk3Nk,2025/10/30
4,sal0005,寵物公園(新城店) - 寵物美容,OPERATIONAL,NTP013,236台灣新北市土城區金城路三段160號1樓,0222735558,63.0,4,3.7,3,2025-10-22,121.467340,24.987179,https://maps.google.com/?cid=12945579740174291962,https://www.wonderpet.asia/,ChIJrzKVgzsDaDQR-hOVYC_wp7M,2025/10/30
5,sal0006,玩美寵物美學館,OPERATIONAL,NTP006,231台灣新北市新店區安民街133巷17號,0222111231,66.0,4,3.7,3,2024-11-11,121.515566,24.969216,https://maps.google.com/?cid=5404861545162166050,https://www.facebook.com/a22111231a/,ChIJSY3RhWwDaDQRIk-Pa_DsAUs,2025/10/30
6,sal0007,希苟，Say Gao狗狗沐浴造型,OPERATIONAL,NTP003,235台灣新北市中和區信義街97號,0229428695,48.0,4,5.0,26,2025-10-09,121.512020,24.987329,https://maps.google.com/?cid=13702982971024496707,https://www.facebook.com/share/1LL5uBDr1D/?mib...,ChIJ8xU3GUIDaDQRQ1gkH2nGKr4,2025/10/30
7,sal0008,波波好室 pet grooming,OPERATIONAL,NaN,231台灣新北市新店区中兴路一段166號1樓,0289117077,45.0,4,5.0,48,2022-11-08,121.542816,24.964147,https://maps.google.com/?cid=2154110100257467416,http://www.facebook.com/bogiihouse,ChIJZQxC0tIBaDQRGGztJLnv5B0,2025/10/30
8,sal0009,OBO寵物會館,OPERATIONAL,NTP006,231台灣新北市新店區復興路44之4號1樓,0229138616,45.0,4,4.8,57,2024-06-08,121.538743,24.986919,https://maps.google.com/?cid=16262247115821394058,https://www.facebook.com/petclubhouse/,ChIJBckxcf8BaDQRiqgnPJcbr-E,2025/10/30
9,sal0010,寵物公園(新店民權店) - 專業寵物用品&寵物美容,OPERATIONAL,NTP006,231007台灣新北市新店區民權路2號,0229106679,77.0,4,5.0,663,2024-09-26,121.543110,24.982995,https://maps.google.com/?cid=9735947480668024293,https://www.wonderpet.asia/,ChIJ8yNsyRYBaDQR5a0TgaYIHYc,2025/10/30


In [48]:
df_final.to_sql(name="salon", con=engine, index=False, if_exists="replace")

11